# Modelling and analyzing a simple unbalanced network with Roseau Load Flow solver

### Introduction

This tutorial is meant to introduce the procedure for modelling components of a given low voltage (LV) network using 
the _Roseau Load Flow_ solver. It will also cover the process of running a power flow, accessing results, and analysis 
such as voltage regulation and energy losses.

Consider the simple LV network shown in the figure below containing a MV/LV, $\Delta$-Y transformer (20kV/0.4kV, 
250 kVA) between the source bus and bus A, a 240 mm², 3-phase line connecting buses A and B, and three 16 mm² 
single-phase lines connecting bus B with buses C, D and E each of which serves as a connection point for a house.

<center> <img style="float: middle;" 
          src="../images/LV_Network_With_Neutral.png" 
          alt="Simple LV network"
          width="40%"> 
</center>

**<center> Figure 1. Simple LV Network</center>**


### Import the required modules and functions

First we need to import the modules and functions needed for this tutorial. The purpose of the following code are 
explained as follows:

- `import roseau.load_flow as rlf `: imports the Roseau Load Flow package. This package is needed to model the 
  various components of the network such as buses, lines, transformers, etc.

- `import numpy as np`: imports the _numpy_ package which will be used for manipulating parameter arrays, calculating 
  exponents, and for retrieving the pi constant.


In [ ]:
import numpy as np
import roseau.load_flow as rlf

### Creating the buses

After importing all the necessary modules, we can move on to the modelling of the given LV network. We start with the 
buses which are the nodes of the network. To create a bus in _Roseau Load Flow_ (RLF), we use the 
[`rlf.Bus`](https://roseau-load-flow.roseautechnologies.com/models/Bus.html) class and pass the constructor an 
identifier and the number of phases that the bus has.

The identifier is used internally by the solver to identify network elements and is also used to name the element when 
displaying results. Almost all elements in _RLF_ require an id when instantiating. Also, a bus in _RLF_ can either be 
a three-phase bus (with or without neutral i.e. `abcn` or `abc`) or any combination of one/two phases with or 
without the neutral. More information on buses and possible phase combinations in _RLF_ can be found 
[here](https://roseau-load-flow.roseautechnologies.com/models/Bus.html).

For this tutorial, the network's buses are created as follows:

- `source_bus = rlf.Bus(id='source_bus', phases='abc')`: this creates a 3-phase source bus with no neutral since it's 
  typically not needed in a transmission network.
- `bus_a = rlf.Bus(id="bus_A", phases="abcn")`: this creates bus A as a 3-phase bus with a neutral (since it's a 
  distribution network bus). Bus B is created in a similar manner.
- `bus_c = rlf.Bus(id="bus_C", phases="an")`: this creates bus C as a single phase bus with a neutral. Buses D and E 
  are created in a similar manner but with different phases.


In [ ]:
source_bus = rlf.Bus(id="source_bus", phases="abc")
bus_a = rlf.Bus(id="bus_A", phases="abcn")
bus_b = rlf.Bus(id="bus_B", phases="abcn")
bus_c = rlf.Bus(id="bus_C", phases="an")
bus_d = rlf.Bus(id="bus_D", phases="bn")
bus_e = rlf.Bus(id="bus_E", phases="cn")

### Adding a voltage source to the source bus

Because the source bus is responsible for powering the network, we need to add a voltage source at the bus to reflect 
this. To add a voltage source to a bus in _RLF_, we first need to set its voltage value. Since the source bus is 
a 3-phase bus, we need a set of three-phase voltages (one for each phase).

A voltage source can either be $\Delta$ or Y-connected as described 
[here](https://roseau-load-flow.roseautechnologies.com/models/VoltageSource.html#connections). Since the source bus 
has no neutral, therefore, we can only add a $\Delta$-connected voltage source to it. The complete procedure is as 
follows:

1. `v_delta = 20e3 * np.exp([0, -2j * np.pi / 3, 2j * np.pi / 3])`: creates a complex array of three-phase voltages with 
  a magnitude of 20 kV and a phase shift of 120 degrees between them.

2. Next, we create the voltage source itself using the 
  [`rlf.VoltageSource`](https://roseau-load-flow.roseautechnologies.com/models/VoltageSource.html) class and specify an 
  id, the bus it is connected to and its voltages:
  `vs = VoltageSource(id="vs", bus=source_bus, voltages=v_delta)`


In [ ]:
v_delta = 20e3 * np.exp([0, -2j * np.pi / 3, 2j * np.pi / 3])
vs = rlf.VoltageSource(id="vs", bus=source_bus, voltages=v_delta)

In a similar manner, it would be possible to provide only a scalar to the `voltages` argument: `voltages=20e3`. As 
the bus is a three-phase bus, the voltages would have been automatically converted to the array `v_delta`. 

### Adding the transformer

Next, we will add the MV/LV transformer. To add a transformer in _RLF_, first we need to define the transformer's 
parameters using the 
[`rlf.TransformerParameters`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) class. 
This can be done in multiple ways in _RLF_ as described 
[here](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html)

For the purpose of this tutorial, we will be using predefined transformer parameters from the catalogue of transformers
included in _RLF_. To do this, we use the `from_catalogue` method of the 
[`rlf.TransformerParameters`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) class. In 
this case, we use the parameters of a 250 kVA Schneider Electric Minera transformer with the highest efficiency (AA0Ak)
as shown in the code cell below. More information on retrieving transformer parameters from the catalogue can be 
seen [here](https://roseau-load-flow.roseautechnologies.com/usage/Catalogues.html#transformers)


In [ ]:
tp = rlf.TransformerParameters.from_catalogue("SE_Minera_AA0Ak_250kVA")

Once the transformer's parameters have been defined, a transformer can then be created using the 
[`rlf.Transformer`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) class. We pass in 
the id of the transformer as well as the buses and phases on its primary side (1) and secondary side (2). Finally, we 
pass in the transformer's parameters created above.


In [ ]:
transformer = rlf.Transformer(id="transf", bus1=source_bus, bus2=bus_a, phases1="abc", phases2="abcn", parameters=tp)

### Adding potential references and grounds

To run power flow calculations in _RLF_, a potential reference must be added to an element in each galvanically 
isolated section of the network. For this tutorial, the isolated sections are the primary and secondary sides of the 
transformer.

For the primary side of the transformer, we use a 
[`rlf.PotentialRef`](https://roseau-load-flow.roseautechnologies.com/models/PotentialRef.html) connected to the 
`source_bus`. We pass in its identifier, the target `element` (which can be a `Bus` or a `Ground`) whose potential 
will be set to 0 V and an optional `phase` argument which is set to `None` by default. In this case, as the target bus 
`source_bus` has three potentials (`a`, `b` and `c`) and as the `phase` argument is not defined, the sum of these 
potentials is set to 0 V by `pref_mv`.


In [ ]:
pref_mv = rlf.PotentialRef(id="pref_mv", element=source_bus)

For the LV part of the network, we will be using lines with shunt connections. These lines will require a 
[`rlf.Ground`](https://roseau-load-flow.roseautechnologies.com/models/Ground.html) element which can be created as 
shown below. The constructor of this element only takes an id.


In [ ]:
ground = rlf.Ground(id="ground")

Now, we can connect this ground to the neutral of the secondary side of the transformer using its `connect` method.


In [ ]:
ground.connect(bus=bus_a, phase="n")

For now, the potential of the element `ground` has not been set to 0 V. In _RLF_, the ability to define `Ground` 
elements has been separated from the ability to set potentials to zero. This is a unique feature of _RLF_ and will 
become more familiar with subsequent tutorials.

In order to set the potential of the element `ground` to 0 V, we just create another 
[`rlf.PotentialRef`](https://roseau-load-flow.roseautechnologies.com/models/PotentialRef.html).


In [ ]:
pref_lv = rlf.PotentialRef(id="pref_lv", element=ground)

Here, the element provided to the constructor is a `Ground` element. As a `Ground` has a single potential, the 
`phase` argument of the constructor must be ignored.


### Adding Lines

Next, we'll add in all the lines present in the network. Similar to the transformer, to add lines in _RLF_, we first 
need to specify the parameters for the line and this can be done manually using the 
[`rlf.LineParameters`](https://roseau-load-flow.roseautechnologies.com/models/Line/Parameters.html) or with the 
catalogue. For manual entry, we will need to specify the series impedance matrix (`z_line`) and optionally, the shunt 
admittance matrix (`y_shunt`). The dimensions of these matrices must be equal to the number of conductors.

Just as we did previously, we will use the _RLF_ catalogue to retrieve predefined parameters for lines. To do this we 
pass in the name of the desired line parameter as well as the number of phases (conductors) required which can range 
from 1 to 4. For the three-phase line (with neutral) between buses A and B, we will use parameters from an overhead 
aluminium cable with a cross-sectional area of 240 mm². The single phase lines will be modelled using parameters 
from an underground aluminium cable with a cross-sectional area of 19 mm² as shown below. Using line parameters make 
it easy to create multiple lines with the same characteristics.

- `lp_240 = rlf.LineParameters.from_catalogue("O_AL_240", nb_phases=4)`
- `lp_19 = rlf.LineParameters.from_catalogue("U_AL_19", nb_phases=2)`

Next, we create the actual lines using the 
[`rlf.Line`](https://roseau-load-flow.roseautechnologies.com/models/Line/index.html) class. To do this, we pass the 
constructor the line's identifier, the buses it's connected to (bus 1 and bus 2), length of the line, number of 
phases, and the line's parameters as shown below. In _RLF_, when the `y_shunt` matrix is given in the line parameters, 
we must define a ground to be used for shunt connections. Here, we reuse the same ground for the LV part of the network.

- `line_ab = rlf.Line(id="lineA_B", bus1=bus_a, bus2=bus_b, parameters=lp_240, phases="abcn", length=rlf.Q_(1.0, "km"), ground=ground)`

**Note**: To demonstrate the flexibility of _RLF_, the length of the line may be specified in multiple units of 
distance such as kilometer, miles, meters, etc using the `rlf.Q_` (Quantity) class. This class takes in the numerical 
value of the length and its unit.

However, it should also be noted that most arguments in _RLF_ functions, methods or classes can also be given without 
using the `rlf.Q_` constructor. In this case the default unit used by that function, class or method will be assumed.


In [ ]:
# Getting line parameters from the catalogue
lp_240 = rlf.LineParameters.from_catalogue("O_AL_240", nb_phases=4)
lp_19 = rlf.LineParameters.from_catalogue("U_AL_19", nb_phases=2)


# Creating the actual lines
line_ab = rlf.Line(
    id="lineA_B", bus1=bus_a, bus2=bus_b, parameters=lp_240, phases="abcn", length=rlf.Q_(1.0, "km"), ground=ground
)
line_bc = rlf.Line(
    id="lineB_C", bus1=bus_b, bus2=bus_c, phases="an", parameters=lp_19, length=rlf.Q_(10, "m"), ground=ground
)
line_bd = rlf.Line(
    id="lineB_D", bus1=bus_b, bus2=bus_d, phases="bn", parameters=lp_19, length=rlf.Q_(10, "m"), ground=ground
)
line_be = rlf.Line(
    id="lineB_E", bus1=bus_b, bus2=bus_e, phases="cn", parameters=lp_19, length=rlf.Q_(10, "m"), ground=ground
)

### Adding loads

In the final modelling step, we will add the loads at buses C, D and E. The description of the loads are given in the 
table below. _RLF_ can model different types of loads as described 
[here](https://roseau-load-flow.roseautechnologies.com/models/Load/index.html).

| Load Name | Phases | Connected bus | Peak Demand (kW) | PF   |
| :-------- | :----- | :------------ | :--------------- | :--- |
| Load_1    | 1      | C             | 7                | 0.95 |
| Load_2    | 1      | D             | 6                | 0.95 |
| Load_3    | 1      | E             | 8                | 0.95 |

For this tutorial, the loads are assumed to be drawing constant power irrespective of the voltage. This means we can 
model them using a [`rlf.PowerLoad`](https://roseau-load-flow.roseautechnologies.com/models/Load/PowerLoad.html) class.
 This class takes in the id of the load, the bus it is connected to, which phase of that bus it is attached to, and the 
 apparent power of the load in complex number notation. The power must be given as an array of one or three values for 
 single and 3-phase loads respectively.

Since we are given active power and power factor instead of apparent power, then we need to carry out a conversion. For
 this we have written a small function, `convert_power` that takes in the peak demand and the power factor to carry out
  the calculation below.

$$\underline{S} = P \left[1 + j\tan\left(\cos^{-1}(pf)\right)\right]$$


In [ ]:
def convert_power(power: float, pf: float) -> complex:
    return power * (1 + np.tan(np.arccos(pf)) * 1j)

Then, we can finally create the load as follows:


In [ ]:
load1 = rlf.PowerLoad(id="load1", bus=bus_c, phases="an", powers=[convert_power(7e3, 0.95)])
load2 = rlf.PowerLoad(id="load2", bus=bus_d, phases="bn", powers=[convert_power(6e3, 0.95)])
load3 = rlf.PowerLoad(id="load3", bus=bus_e, phases="cn", powers=[convert_power(8e3, 0.95)])

Note that these three loads are connected between a phase and the neutral of their bus which is defined using the same 
phases. In cases like this where the phases of the bus and the load are the same specifying the phases in the 
`PowerLoad` constructor is a bit redundant as the phases of the buses are taken by default. Furthermore, the `powers`
 argument must be an array of length equal to the number of bus phases.


### Building the network

Following the modelling of all network components, the next step is to build the actual network. Currently, components 
only exist in isolation and they do not belong to any network. This can be verified by running the code below which 
returns nothing.


In [ ]:
load1.network is None

To build the network in _RLF_, we use the `rlf.ElectricalNetwork` class. This class provides several methods for 
constructing the network and perhaps the easiest one is the `from_element` method. This method allows you to create 
the entire network from a single bus and adds all the other elements automatically. So, we pass in the source bus to 
create the entire network as shown below.


In [ ]:
en = rlf.ElectricalNetwork.from_element(initial_bus=source_bus)

To verify if the network has been successfully created, we can just print `en` to see the number of elements in the built network.


In [ ]:
en

### Running a load flow

To execute load flow calculations in _RLF_, a license is required. However, a free, limited license key is available [`here`](https://roseau-load-flow.roseautechnologies.com/License.html#license-types) and can be used for the purpose of this tutorial.

Once the license is properly installed, we can simply run a load flow as shown below:


In [ ]:
en.solve_load_flow()

This function returns the number of iterations performed by the solver and the residual error after convergence.


### Accessing results

After running the load flow calculations, we can extract the value of several quantities from different network elements. Each quantity can be accessed through the name of the quantity prefixed with the phrase `res_`. A complete breakdown of possible results for each network element is given below.

| Element type                                | Available results                                                                                                                                       |
| ------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `Bus`                                       | `res_potentials`, `res_voltages`, `res_violated`                                                                                                        |
| `Line`                                      | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_series_power_losses`, `res_shunt_power_losses`, `res_power_losses`, `res_violated` |
| `Transformer`                               | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_power_losses`, `res_violated`                                                      |
| `Switch`                                    | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`                                                                                          |
| `ImpedanceLoad`, `CurrentLoad`, `PowerLoad` | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`, `res_flexible_powers`&#8270;                                                            |
| `VoltageSource`                             | `res_currents`, `res_powers`, `res_potentials`, `res_voltages`                                                                                          |
| `Ground`                                    | `res_potential`                                                                                                                                         |
| `PotentialRef`                              | `res_current` _(Always zero for a successful load flow)_                                                                                                |

&#8270;: `res_flexible_powers` is only available for flexible loads (`PowerLoad`s with `flexible_params`).

To access results for different elements in _RLF_, there are two main methods. The first method involves calling the `res_` method for the desired quantity for that element e.g.


In [ ]:
load1.res_voltages

Note that the return type is a Quantity (`rlf.Q_`) class. This is the same when accessing any element's results using 
the `res_` method. To retrieve the actual numerical value of the quantity, we can just use the `.m` method of the 
`rlf.Q_` class as shown below.


In [ ]:
load1.res_voltages.m

However, this method of accessing results can only be used for one element at a time. To extract results for multiple 
similar elements, we can call the `res_` method of the electrical network instead e.g.


In [ ]:
en.res_loads

We get a `DataFrame` (from the `Pandas` library) of results. In a data frame, the units are not included thus the values are always displayed in SI units of V, A and W.


##### Accessing active and reactive power of loads

Although, these were explicitly specified, we can still verify the results. We will apply both methods described above 
to retrieve the active (P) and reactive (Q) powers of the three loads present in the network. Note the `res_power` 
method in _RLF_ returns the powers in the live and neutral conductors. Therefore, the total power dissipated by the 
load can be determined by taking a sum of these values as shown below.


In [ ]:
# Method 1
load_power = load1.res_powers.m_as("kVA").sum()
print(f"Active power of Load 1: {round(load_power.real, 3)} kW")
print(f"Reactive power of Load 1: {round(load_power.imag, 3)} kVar")

Here, the result is converted to kVA using the `m_as` method of the quantities (`Q_`) class.


In [ ]:
# Method 2
# We extract the DataFrame of results
load_powers = en.res_loads["power"].sort_index()

# We extract the active and reactive power of each load
for index in load_powers.index.get_level_values(0).unique():
    load = load_powers.loc[index].sum()
    print(f"Active power of Load 1: {round(load.real/1e3, 3)} kW")
    print(f"Reactive power of Load 1: {round(load.imag/1e3, 3)} kVar")

##### Voltage magnitude of the load and 3-phase buses

Similar to the loads, we can extract the voltage magnitude of a certain bus using its `res_voltages` property. 
This returns the voltage in complex form which can be converted into polar form to extract the magnitude and phase angle.

To extract the results for multiple or all buses simultaneously, we can use the `res_buses` property of the electrical
 network. This returns a dataframe which can be filtered to get the results for the desired buses. By default, the 
 results are also in rectangular format. To convert to polar form, we use the transform method of dataframes and apply 
 two functions to get the magnitude and phase angle (in degrees) of the voltage.

Both methods are shown below.


In [ ]:
# Method 1

# Voltage magnitude at bus C
np.abs(bus_c.res_voltages)

In [ ]:
# Voltage angle at bus C (in degrees)
np.angle(bus_c.res_voltages.m, deg=True)

In [ ]:
# Method 2
import functools as ft

en.res_buses_voltages["voltage"].transform([np.abs, ft.partial(np.angle, deg=True)])

#### Active and Reactive powers of the transformer

Similarly, to extract the active and reactive powers for the transformer, we can use the two methods of accessing 
results as shown below. It can be observed that the powers in the secondary side of the transformer are negative. 
This indicates that the power is leaving the transformer as opposed to the positive powers on the primary side 
entering the transformer. Also, there is no power flowing thorough the neutral phase as expected.


In [ ]:
# Method 1

# Primary Side Powers (in kVA and rounded)
np.round(transformer.res_powers[0].to("kVA"), 3)

In [ ]:
# Secondary Side Powers (in kVA and rounded)
np.round(transformer.res_powers[1].to("kVA"), 3)

In [ ]:
# Method 2 (in kVA and rounded)
np.round(en.res_transformers[["power1", "power2"]] * 1e-3, 3)

In addition, we can also retrieve the power lost in the transformer using the `res_power_losses` property of the [`rlf.Transformer`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) object as shown below.


In [ ]:
transformer.res_power_losses.m

#### Line Losses

Finally, the line losses on the network can also be conveniently accessed using the described methods. The first method uses the `res_series_power_losses` of each line and returns the line losses for each phase. The second method uses the `res_lines` property of the electrical network and the column `series_losses` in the resulting dataframe contains the power losses on each phase. Conventionally, when talking about line losses, we are typically interested in the active power losses in the lines which can be obtained as shown below.


In [ ]:
# Method 1 (unit: W)
np.real(line_ab.res_series_power_losses.m)

In [ ]:
# Method 2 (unit: W)
en.res_lines["series_losses"].transform([np.real])